In [1]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn.metrics as metrics
import seaborn as sns
from pathlib import Path 

In [15]:
import warnings
warnings.filterwarnings("ignore")

In [20]:
from pathlib import Path
p = Path().cwd()
print(f"original File path: {p}")
stringpath= str(p)[0:str(p).rfind('\\')] + '\\data'
print(f"Updated File Path to pick from Data Library: { stringpath}")
ngsimfile=stringpath + '/' + 'Cleaned_NGSIM_Data_Part_11.csv'
ngsim = pd.read_csv(ngsimfile,low_memory=False)
print(f"{ngsim.duplicated().sum()} duplicate values have been removed")
ngsim.drop_duplicates(inplace=True)

original File path: c:\Users\neels\car following model\DataDrivenCarFollowing-1\datadrivencarfollowing-v1\scripts
Updated File Path to pick from Data Library: c:\Users\neels\car following model\DataDrivenCarFollowing-1\datadrivencarfollowing-v1\data
0 duplicate values have been removed


In [21]:
ngsim.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8654872 entries, 0 to 8654871
Data columns (total 20 columns):
 #   Column                       Dtype  
---  ------                       -----  
 0   Vehicle_ID                   int64  
 1   Frame_ID                     int64  
 2   Global_Time                  int64  
 3   Local_X                      float64
 4   Local_Y                      float64
 5   Global_X                     float64
 6   Global_Y                     float64
 7   v_length                     float64
 8   v_Class                      int64  
 9   v_Vel                        float64
 10  v_Acc                        float64
 11  Preceding                    int64  
 12  Space_Headway                float64
 13  Time_Headway                 float64
 14  Location                     object 
 15  Relative_Time                int64  
 16  Preceding_Vehicle_Class      object 
 17  Rear_to_Front_Space_Headway  float64
 18  v_Class_Name                 object 
 19  

In [22]:
df = ngsim[ngsim["Location"] == "i-80"]

In [23]:
df["Vehicle_ID"]=df["Vehicle_ID"].astype(str)
df["Preceding"]=df["Preceding"].astype(str)
df["LF_pairs"] = df["Vehicle_ID"]+ "_"+df["Preceding"]

In [24]:
df["Time.per.frame"] = 0.1

In [25]:
df_grp = df.groupby(["LF_pairs"])

In [27]:
#Time duration per pair of each time frame.
df['Time_duration']= df.groupby(["LF_pairs"], as_index=False)["Frame_ID"].cumcount()*0.1

In [28]:
new_column = df.groupby(["LF_pairs"], as_index=False).apply(lambda x : len(x["Frame_ID"]) * 0.1 )
df = pd.merge(df, new_column, how="inner", on=["LF_pairs"])

In [29]:
df["nextframeAcc"]=df.groupby(["LF_pairs"],as_index=False)["v_Acc"].shift(-1)

In [30]:
df["nextframesvel"]=df.groupby(["LF_pairs"],as_index=False)["v_Vel"].shift(-1)

In [31]:
df["nextframeposition"]=df.groupby(["LF_pairs"],as_index=False)["Local_Y"].shift(-1)

In [32]:
df

,Vehicle_ID,Frame_ID,Global_Time,Local_X,Local_Y,Global_X,Global_Y,v_length,v_Class,v_Vel,...,Rear_to_Front_Space_Headway,v_Class_Name,Vehicle_combination,LF_pairs,Time.per.frame,Time_duration,None,nextframeAcc,nextframesvel,nextframeposition
0,2224,6548,1113437421700,41.429,472.901,6042814.264,2133542.012,4.35864,2,8.089392,...,11.736832,Car,Car-Car,2224_2208,0.1,0.0,128.0,0.000000,0.000000,1084.757
1,2224,7548,1113437521700,42.424,1084.757,6042732.771,2134149.721,4.35864,2,0.000000,...,2.537968,Car,Car-Car,2224_2208,0.1,0.1,128.0,0.000000,0.000000,811.735
2,2224,6901,1113437457000,39.751,811.735,6042769.994,2133878.430,4.35864,2,0.000000,...,1.120648,Car,Car-Car,2224_2208,0.1,0.2,128.0,0.027432,3.761232,184.565
3,2224,6390,1113437405900,41.500,184.565,6042849.684,2133255.972,4.35864,2,3.761232,...,3.714496,Car,Car-Car,2224_2208,0.1,0.3,128.0,0.000000,3.276600,198.257
4,2224,6402,1113437407100,41.507,198.257,6042848.002,2133269.560,4.35864,2,3.276600,...,5.332984,Car,Car-Car,2224_2208,0.1,0.4,128.0,0.000000,0.000000,960.083
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4555934,1498,4892,1113433624100,48.681,1512.818,6042670.346,2134573.643,3.47472,2,3.617976,...,7.887208,Car,Car-Car,1498_1468,0.1,0.0,0.1,NaN,NaN,NaN
4555935,2313,7589,1113433893800,36.269,1576.328,6042648.528,2134630.789,3.62712,2,5.763768,...,0.222504,Car,Car-Car,2313_2316,0.1,0.0,0.1,NaN,NaN,NaN
4555936,1341,5164,1113438081300,65.138,533.449,6042830.386,2133605.071,5.42544,2,5.090160,...,30.345888,Car,Car-Car,1341_1310,0.1,0.0,0.1,NaN,NaN,NaN
4555937,508,2767,1113437841600,59.864,715.289,6042802.481,2133785.021,4.81584,2,8.519160,...,8.598408,Car,Car-Car,508_506,0.1,0.0,0.1,NaN,NaN,NaN


In [33]:
df["LF_pairs"].value_counts()

2057_2054    3669
1848_1841    2950
1946_1940    2715
1833_1825    2484
2488_2475    2467
             ... 
1922_1854       1
1163_1113       1
1138_1188       1
745_731         1
1867_1969       1
Name: LF_pairs, Length: 15095, dtype: int64

In [57]:
df1 = df.tail(1000)

In [53]:
from millify import millify
pre_Vel = []

for i in df1.index:
    Vel =df1['v_Vel'][(df1['Relative_Time'] == df1['Relative_Time'][i])|(df1['Vehicle_ID']== df1['Preceding'][i])].mean()
    pre_Vel.append(Vel)
    if i%500000==0:
        print(f"row count{millify(i, precision=2, drop_nulls=False)} processed")
        

df1['pre_Vel'] = pre_Vel

row count0.00 processed


In [54]:
df1['pre_Vel']

0      8.089392
1      0.000000
2      0.000000
3      3.761232
4      3.276600
         ...   
995    4.117848
996    4.791456
997    4.681728
998    5.943600
999    0.341376
Name: pre_Vel, Length: 1000, dtype: float64

In [58]:
df1.T

,4554939,4554940,4554941,4554942,4554943,4554944,4554945,4554946,4554947,4554948,...,4555929,4555930,4555931,4555932,4555933,4555934,4555935,4555936,4555937,4555938
Vehicle_ID,1468,1468,1468,1468,1468,1468,1468,1468,401,401,...,1163,1163,1226,1226,1226,1498,2313,1341,508,1867
Frame_ID,4342,4352,4355,4345,4354,4344,4349,4358,2400,2401,...,5002,5001,4687,4685,4686,4892,7589,5164,2767,6642
Global_Time,1113437201100,1113437202100,1113437202400,1113437201400,1113437202300,1113437201300,1113437201800,1113437202700,1113437804900,1113437805000,...,1113438065100,1113438065000,1113438033600,1113438033400,1113438033500,1113433624100,1113433893800,1113438081300,1113437841600,1113437431100
Local_X,75.751,79.588,80.69,76.968,80.322,76.556,78.487,81.793,54.504,54.456,...,13.176,12.833,53.496,53.494,53.494,48.681,36.269,65.138,59.864,18.728
Local_Y,422.674,457.267,467.703,432.0,464.219,428.465,446.839,478.02,675.859,678.123,...,1576.818,1571.447,177.026,173.826,175.426,1512.818,1576.328,533.449,715.289,1630.859
Global_X,6042854.504,6042854.032,6042853.867,6042854.421,6042853.922,6042854.476,6042854.197,6042853.703,6042802.097,6042801.766,...,6042625.094,6042625.607,6042862.519,6042862.912,6042862.715,6042670.346,6042648.528,6042830.386,6042802.481,6042622.003
Global_Y,2133497.331,2133531.091,2133541.575,2133506.565,2133538.081,2133503.248,2133520.607,2133552.059,2133745.232,2133747.47,...,2134631.068,2134625.711,2133249.97,2133246.794,2133248.382,2134573.643,2134630.789,2133605.071,2133785.021,2134685.333
v_length,4.54152,4.54152,4.54152,4.54152,4.54152,4.54152,4.54152,4.54152,3.74904,3.74904,...,2.37744,2.37744,4.66344,4.66344,4.66344,3.47472,3.62712,5.42544,4.81584,4.05384
v_Class,2,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
v_Vel,7.260336,10.594848,10.570464,11.423904,10.594848,10.415016,10.594848,10.491216,6.9342,6.986016,...,16.370808,16.370808,4.8768,4.8768,4.8768,3.617976,5.763768,5.09016,8.51916,3.334512
